Importing Libraries


In [1]:
import pandas as pd  # Importing pandas library for data manipulation
import numpy as np  # Importing numpy library for numerical computations
from tensorflow.keras.preprocessing.text import Tokenizer  # Importing Tokenizer for text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Importing pad_sequences for sequence padding
from tensorflow.keras.utils import to_categorical  # Importing to_categorical for one-hot encoding
from sklearn.preprocessing import LabelEncoder  # Importing LabelEncoder for label encoding
from tensorflow.keras.models import Model  # Importing Model for defining neural network models
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense  # Importing layers for building neural networks

Loading the Dataset



In [3]:
# Reading the medical data from a CSV file
data = pd.read_csv('medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


Tokenizing and Sequencing Text Data



Before using medical data in a deep learning model, it needs to be preprocessed to ensure the model can understand it. Preprocessing steps often include:

Text Tokenization: Converting textual data into sequences of numbers that the model can process.

Padding Sequences: Making all sequences the same length by adding padding characters at the beginning or end of shorter sequences.

Label Encoding: Converting categorical variables, such as disease names and medication names, into numerical labels.

A ‘tokenizer’ variable is created to convert the textual data into sequences of integers. It only considers the top 5,000 words in the dataset in order to reduce the complexity. If the model encounters any out-of-vocabulary words during the training process then it will be replaced with the ‘<OOV>’ token.

In [4]:
# Initializing a Tokenizer object with a vocabulary size of 5000 and an out-of-vocabulary token

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")

# Fitting tokenizer on patient problems
tokenizer.fit_on_texts(data['Patient_Problem'])
# Converting text to sequences
sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])

Padding Sequences


Padding Sequences
In order to make the input sequences have the same length, the code finds the longest sequence and pads all other sequences with zeros at the end (‘post’ padding) to match this sentence.

In [5]:
max_length = max(len(x) for x in sequences)  # Finding the maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')  # Padding sequences


Encoding the Labels and Converting them to Categorical
We will encode the ‘Disease’ and ‘Prescription’ columns as integers. Then the integer-encoded labels are converted into binary class matrices.

In [6]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

# Encoding disease and prescription labels
disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

Combining Labels into a Multi-label Target Variable


Combining Labels into a Multi-label Target Variable
Finally, now we will stack the binary class matrices together to form a single multi-label target variable

‘Y’. This allows the model to predict both ‘Disease‘ and ‘Prescription‘ from the patient’s problem

In [7]:
# Concatenating disease and prescription labels

Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))


Model Building


Model Building
Now, we will build the model using the LSTM and Sequential algorithm from TensorFlow. This model will learn from our preprocessed dataset to predict diseases based on patient symptoms.

Defining Model Architecture
We will use the ‘Model’ and ‘Input’ to define the model architecture, and ‘Embedding’ to convert the integer sequences into dense vectors of fixed size. We will use ‘Dense’ for output layers that make predictions.

In [8]:
input_layer = Input(shape=(max_length,))  # Defining input layer

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)  # Adding an embedding layer
lstm_layer = LSTM(64)(embedding)  # Adding LSTM layer

# Output layer for disease prediction
disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax',
                       name='disease_output')(lstm_layer)

# Output layer for prescription prediction
prescription_output = Dense(len(label_encoder_prescription.classes_),
                            activation='softmax', name='prescription_output')(lstm_layer)

Compiling the model



The model firstly have, an input layer that can handle sequences up to a certain length. Then there’s an embedding layer that turns the numbers into vectors. After that, there’s an LSTM layer that looks at the order of things, and finally, two dense layers that predict diseases and prescriptions using a softmax function for classification.

In [9]:
# Defining the model with inputs and outputs

model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

# Compiling the model with appropriate loss functions and optimizers
model.compile(
    loss={'disease_output': 'categorical_crossentropy',
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

# Printing model summary

model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 17, 64)               320000    ['input_1[0][0]']             
                                                                                                  
 lstm (LSTM)                 (None, 64)                   33024     ['embedding[0][0]']           
                                                                                                  
 disease_output (Dense)      (None, 178)                  11570     ['lstm[0][0]']                
                                                                                             

Training the model



In [10]:
# Training the model

model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output':
      prescription_labels_categorical}, epochs=100, batch_size=32)


Epoch 1/100


13/13 [==============================] - 9s 38ms/step - loss: 11.1468 - disease_output_loss: 5.1811 - prescription_output_loss: 5.9657 - disease_output_accuracy: 0.0098 - prescription_output_accuracy: 0.0000e+00
Epoch 2/100
13/13 [==============================] - 1s 39ms/step - loss: 11.1276 - disease_output_loss: 5.1653 - prescription_output_loss: 5.9622 - disease_output_accuracy: 0.0319 - prescription_output_accuracy: 0.0025
Epoch 3/100
13/13 [==============================] - 1s 54ms/step - loss: 11.0996 - disease_output_loss: 5.1358 - prescription_output_loss: 5.9638 - disease_output_accuracy: 0.0270 - prescription_output_accuracy: 0.0025
Epoch 4/100
13/13 [==============================] - 1s 47ms/step - loss: 11.0163 - disease_output_loss: 5.0263 - prescription_output_loss: 5.9900 - disease_output_accuracy: 0.0270 - prescription_output_accuracy: 0.0098
Epoch 5/100
13/13 [==============================] - 1s 42ms/step - loss: 10.8969 - disease_output_loss: 4.9170 - 

Making Predictions


The model is used to make predictions for new patients:

1-Pre-processed the patient’s symptoms by performing tokenization and padding.

2-Feed the pre-processed data into the trained model.

3-The model predicts the disease and medication based on the patient’s symptoms.

4-The predicted disease and medication will be presented.

In [11]:
# Function to make predictions based on patient problem input

def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Making prediction
    prediction = model.predict(padded_sequence)

    # Decoding the prediction
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    # Printing the predicted disease and prescription

    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")


"""I've experienced a loss of appetite and don't enjoy food anymore.
Frequent severe migraines, sensitivity to light and sound.
Sudden weight gain and feeling cold, especially in the hands and feet.
High fever, sore throat, and swollen lymph nodes, feeling very weak.
Sharp, stabbing chest pain that worsens when lying down or inhaling deeply.
Unexplained weight loss and night sweats, feeling feverish.
Severe abdominal pain radiating to the back, nausea without vomiting.
Joint swelling and stiffness in the morning, especially in the fingers and wrists.
Recurrent, painful blisters around the mouth and nose.
Progressive difficulty swallowing, accompanied by heartburn.
Chronic diarrhea, abdominal cramps, and gas.
Yellowing of the skin and eyes, dark urine, and general fatigue."""
#patient_input = "Constant fatigue and muscle weakness, struggling to stay awake."
#make_prediction(patient_input)


# Example input for patient problem

patient_input = input(str())
make_prediction(patient_input)

1/1 [==============================] - 2s 2s/step
Predicted Disease: Depression
Suggested Prescription: Antidepressants; eating nutrient-rich foods.
